<a href="https://colab.research.google.com/github/RohanSaxena14/snapchat_filters/blob/master/snapchat_filters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!wget "https://github.com/italojs/facial-landmarks-recognition/raw/master/shape_predictor_68_face_landmarks.dat"
!wget "http://arunponnusamy.com/files/mmod_human_face_detector.dat"
!wget "https://github.com/RohanSaxena14/snapchat_filters/raw/master/data/mr bean.mp4"
!wget "https://github.com/RohanSaxena14/snapchat_filters/raw/master/data/Eye.png"
!wget "https://github.com/RohanSaxena14/snapchat_filters/raw/master/data/tongue.png"

In [0]:
import cv2
import dlib
import numpy as np
from google.colab.patches import cv2_imshow


cnn_face_detector = dlib.cnn_face_detection_model_v1("/content/mmod_human_face_detector.dat")
face_detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("/content/shape_predictor_68_face_landmarks.dat")

In [0]:
def insert_clipart(frame, points, size, clipart):
  clipart = cv2.resize(clipart, (2*size, 2*size))
  orig_mask = clipart[:,:,3]
  inv_mask = 255 - orig_mask
  clipart = clipart[:, :, 0:3]

  cut = frame[points[1]: points[1] + 2*size, points[0]: points[0] + 2*size, :]
  
  cut = cv2.bitwise_and(cut, cut, mask = inv_mask)
  clipart = cv2.bitwise_and(clipart, clipart, mask = orig_mask)

  cut = cv2.add(cut, clipart)

  frame[points[1]:points[1]+ 2*size, points[0]:points[0]+ 2*size, :] = cut

  return frame

In [0]:
cap = cv2.VideoCapture("mr bean.mp4")

ret, image = cap.read()

fourcc = cv2.VideoWriter_fourcc(*'XVID')

out = cv2.VideoWriter('output.avi',fourcc, 20.0, (image.shape[1], image.shape[0]))

num = 0
eye = cv2.imread("/content/Eye.png", -1)

while cap.isOpened():
    ret, image = cap.read()
    
    num += 1
    print(num)

    if ret == False:
      break
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    faces = cnn_face_detector(image, 1)

    for face in faces:
      '''
      x = face.rect.left()
      y = face.rect.top()
      w = face.rect.right() - x
      h = face.rect.bottom() - y
      
      cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 3)
      '''
      

      points = predictor(gray, face.rect)

      '''
      for i in range(0, 68):
        point = (points.part(i).x, points.part(i).y)

        cv2.circle(image, point, 1, (0, 255, 0), -1)
      '''
      
      p_38 = np.array([int((points.part(37).x + points.part(38).x)/2), int((points.part(37).y + points.part(38).y)/2)])
      p_41 = np.array([int((points.part(40).x + points.part(41).x)/2), int((points.part(40).y + points.part(41).y)/2)])

      p_44 = np.array([int((points.part(43).x + points.part(44).x)/2), int((points.part(43).y + points.part(44).y)/2)])
      p_47 = np.array([int((points.part(46).x + points.part(47).x)/2), int((points.part(46).y + points.part(47).y)/2)])

      p_43 = np.array([points.part(42).x, points.part(42).y])
      p_46 = np.array([points.part(45).x, points.part(45).y])

      p_37 = np.array([points.part(36).x, points.part(36).y])
      p_40 = np.array([points.part(39).x, points.part(39).y])


      eye_vertical_left = np.linalg.norm(p_38 - p_41).astype(int)
      eye_vertical_right = np.linalg.norm(p_44 - p_47).astype(int)

      eye_horizontal_right = np.linalg.norm(p_43 - p_46).astype(int)
      eye_horizontal_left = np.linalg.norm(p_37 - p_40).astype(int)

      #if eye_vertical_left/eye_horizontal_right >= 0.4 and eye_vertical_right/eye_horizontal_right >= 0.4: #if you want to add eyes only when your eyes are wide open
      if True: #when you want to add eyes on all frames
        corner_x = p_37[0] - int(eye_horizontal_left / 2)
        corner_y = p_37[1] - eye_horizontal_left

        points_left_eye = (corner_x, corner_y)

        image = insert_clipart(image, points_left_eye, eye_horizontal_left, eye)

        corner_x = p_43[0] - int(eye_horizontal_right / 2)
        corner_y = p_43[1] - eye_horizontal_right

        points_right_eye = (corner_x, corner_y)

        image = insert_clipart(image, points_right_eye, eye_horizontal_right, eye)  
    
    out.write(image) 

cap.release()
out.release()